<a href="https://colab.research.google.com/github/IliyaFrahani/SimpleTextGeneration/blob/main/TXT_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***text generation using keras and tensorflow*** 
* data set :
    https://drive.google.com/file/d/1nhThs5ya5KQ77Nxyghm_Aoi4fo06zWYI/view?usp=share_link
    
    https://drive.google.com/file/d/1dtmFb8dxjVRS_-pI6dse8nngu0JmwGBO/view?usp=share_link

    https://drive.google.com/file/d/1Ca4gXHAfCRzhTnm2f0FSM6OwVq5N5kmc/view?usp=share_link

    https://drive.google.com/file/d/17JQzRfbh7Odh16IjZdRhl3DlDpC1ISCy/view?usp=share_link

# *Write by iliya farahani*

* data sets lang :
  
  persian

  english

# ***Importing Library's :***

In [ ]:
# import lib

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
import json
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense , Dropout
from keras.callbacks import TensorBoard, EarlyStopping


# ***load random sentences json file dataset :***

In [ ]:
# load json file
data = open("/content/first-rows.json")
data = json.load(data)
data = (data["rows"])

# func for cleaning data
def clean(text):
    text = re.sub('[0-9]+.\t','',str(text))
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    text = re.sub("'s",'',str(text))
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    text = re.sub('\"','',str(text))
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    return text

# extrac just anserws from questions and anserws
just_anserws = ""
for i in data : 
  for j in (re.split("Human:", i["row"]["texts"]))[1::] : 
      j = clean(j)
      just_anserws += j.split("Assistant:")[1] + " "
data = just_anserws


# ***Making the data easier for the model :***


In [ ]:
# create a tokenizer and fit it on training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# convert texts to sequences
encoded_text = tokenizer.texts_to_sequences([data])[0]

Hi my name is iliya

# normalize the data
tf.keras.utils.normalize(encoded_text)

# define the length of input sequences
seq_length = 10

input_sequences = []
output_sequences = []
for i in range(seq_length, len(encoded_text)):
    input_sequences.append(encoded_text[i-seq_length:i])
    output_sequences.append(encoded_text[i])
    

# ***enable tracking experiment metrics :***

In [ ]:
# enable tracking experiment metrics

tb_callback = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)
earlystop_callback = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=1, mode='auto')


# ***make nural network model :***

In [ ]:
# define the size of vocabulary
vocab_size = len(tokenizer.word_index) + 1

# define the model for training on data
model = Sequential()
model.add(Embedding(vocab_size , 10 , input_length = seq_length))
model.add(Dropout(0.3))
model.add(LSTM(250, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(250))
model.add(Dropout(0.3))
model.add(Dense(vocab_size , activation = "softmax"))


In [ ]:
# compile the model for checking the format errors
model.compile(loss='sparse_categorical_crossentropy',metrics = ["accuracy"], optimizer='adam')


# ***Fit the data on model and start training :***

In [ ]:
model.fit(input_sequences , output_sequences , epochs = 100 ,callbacks=[tb_callback, earlystop_callback])

Epoch 1/100
389/389 [==============================] - ETA: 0s - loss: 6.6454 - accuracy: 0.0452

389/389 [==============================] - 44s 103ms/step - loss: 6.6454 - accuracy: 0.0452
Epoch 2/100
389/389 [==============================] - ETA: 0s - loss: 6.4874 - accuracy: 0.0465

389/389 [==============================] - 40s 102ms/step - loss: 6.4874 - accuracy: 0.0465
Epoch 3/100
389/389 [==============================] - ETA: 0s - loss: 6.4204 - accuracy: 0.0502

389/389 [==============================] - 40s 102ms/step - loss: 6.4204 - accuracy: 0.0502
Epoch 4/100
389/389 [==============================] - ETA: 0s - loss: 6.3593 - accuracy: 0.0527

389/389 [==============================] - 39s 100ms/step - loss: 6.3593 - accuracy: 0.0527
Epoch 5/100
389/389 [==============================] - ETA: 0s - loss: 6.3052 - accuracy: 0.0577

...

389/389 [==============================] - 38s 97ms/step - loss: 1.4204 - accuracy: 0.6296
Epoch 97/100
389/389 [==============================] - ETA: 0s - loss: 1.4008 - accuracy: 0.6381

389/389 [==============================] - 37s 96ms/step - loss: 1.4008 - accuracy: 0.6381
Epoch 98/100
389/389 [==============================] - ETA: 0s - loss: 1.3506 - accuracy: 0.6476

389/389 [==============================] - 39s 99ms/step - loss: 1.3506 - accuracy: 0.6476
Epoch 99/100
389/389 [==============================] - ETA: 0s - loss: 1.3397 - accuracy: 0.6538

389/389 [==============================] - 38s 97ms/step - loss: 1.3397 - accuracy: 0.6538
Epoch 100/100
389/389 [==============================] - ETA: 0s - loss: 1.3219 - accuracy: 0.6571

389/389 [==============================] - 38s 98ms/step - loss: 1.3219 - accuracy: 0.6571


In [ ]:
# show tensorboard window

%load_ext tensorboard
%tensorboard --logdir ./logs

# ***Predict and generate seed text :***

In [ ]:
def gnr_txt(model , tok ,len_seq ,seed_text , num_tokns) :

   # convert seed text to sequences 
   encoded_seed = tok.texts_to_sequences([seed_text])[0]

   # check encoded text 
   if len(encoded_seed) > len_seq : 
     encoded_seed = encoded_seed[-len_seq:]
   else : 
     encoded_seed = [0] * (len_seq -len(encoded_seed) ) + encoded_seed

   # predict the next word
   for _ in range(num_tokns):
        seed_sequence = np.reshape(encoded_seed, (1, seq_length))
        prediction = model.predict(seed_sequence , use_multiprocessing=True)
        next_token = np.argmax(prediction)
        encoded_seed.append(next_token)
        encoded_seed = encoded_seed[1:]

    # convert sequences to texts
   generated_text = tokenizer.sequences_to_texts([encoded_seed])[0]
    
   return generated_text

In [ ]:
# The next word of the text we want to be specified
seed_txt = "i like to know who "

num_tkn = 10
final_res = gnr_txt(model , tokenizer , seq_length , seed_txt ,num_tkn)
print(seed_txt + final_res)